In [20]:
# import the required libraries 
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
import pickle 
import os.path 
import base64 
import email 
from bs4 import BeautifulSoup 

# Define the SCOPES. If modifying it, delete the token.pickle file. 
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly'] 

def getEmails(): 
    # Variable creds will store the user access token. 
    # If no valid token found, we will create one. 
    creds = None

    # The file token.pickle contains the user access token. 
    # Check if it exists 
    if os.path.exists('token.pickle'): 

        # Read the token from the file and store it in the variable creds 
        with open('token.pickle', 'rb') as token: 
            creds = pickle.load(token) 

    # If credentials are not available or are invalid, ask the user to log in. 
    if not creds or not creds.valid: 
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request()) 
        else: 
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES) 
            creds = flow.run_local_server(port=0) 

        # Save the access token in token.pickle file for the next run 
        with open('token.pickle', 'wb') as token: 
            pickle.dump(creds, token) 

    # Connect to the Gmail API 
    service = build('gmail', 'v1', credentials=creds) 

    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 

    # We can also pass maxResults to get any number of emails. Like this: 
    # result = service.users().messages().list(maxResults=200, userId='me').execute() 
    messages = result.get('messages') 

    # messages is a list of dictionaries where each dictionary contains a message id. 

    # iterate through all the messages 
    for msg in messages: 
        # Get the message from its id 
        txt = service.users().messages().get(userId='me', id=msg['id']).execute() 

        # Use try-except to avoid any Errors 
        try: 
            # Get value of 'payload' from dictionary 'txt' 
            payload = txt['payload'] 
            headers = payload['headers'] 

            # Look for Subject and Sender Email in the headers 
            for d in headers: 
                if d['name'] == 'Subject': 
                    subject = d['value'] 
                if d['name'] == 'From': 
                    sender = d['value'] 

            # The Body of the message is in Encrypted format. So, we have to decode it. 
            # Get the data and decode it with base 64 decoder. 
            parts = payload.get('parts')[0] 
            data = parts['body']['data'] 
            data = data.replace("-","+").replace("_","/") 
            decoded_data = base64.b64decode(data) 

            # Now, the data obtained is in lxml. So, we will parse 
            # it with BeautifulSoup library 
            soup = BeautifulSoup(decoded_data , "lxml") 
            body = soup.body() 

            # Printing the subject, sender's email and message 
            print("Subject: ", subject) 
            print("From: ", sender) 
            print("Message: ", body) 
            print('\n') 
        except: 
            pass


getEmails()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=993625018949-omkq60uonaou2klikkjtvrgckrvleiol.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64132%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=SDSEUVmPZfZUAHwkJezNO5cskmenIT&access_type=offline


In [22]:
if os.path.exists('token.pickle'): 

        # Read the token from the file and store it in the variable creds 
        with open('token.pickle', 'rb') as token: 
            creds = pickle.load(token) 


In [49]:
service = build('gmail', 'v1', credentials=creds)

In [50]:
result = service.users().messages().list(maxResults=10, userId='me').execute()

In [51]:
result

{'messages': [{'id': '176fb9835206461d', 'threadId': '176fb9835206461d'},
  {'id': '176fb832b6bb3de3', 'threadId': '176fb832b6bb3de3'},
  {'id': '176f205dbc18f926', 'threadId': '176f205dbc18f926'},
  {'id': '176f1e6dc62cf4f4', 'threadId': '176f1e6dc62cf4f4'},
  {'id': '176ec926138870ae', 'threadId': '176ec926138870ae'},
  {'id': '1762d6b7cc0b0ee9', 'threadId': '1762d6b7cc0b0ee9'},
  {'id': '175d1229964f9026', 'threadId': '175d1229964f9026'},
  {'id': '175d121424d0d065', 'threadId': '175d121424d0d065'},
  {'id': '17447daaee2b5104', 'threadId': '17447daaee2b5104'},
  {'id': '172c66b55df6503b', 'threadId': '172c66b55df6503b'}],
 'nextPageToken': '05473480117891521722',
 'resultSizeEstimate': 11}

In [52]:
messages=result.get('messages') 

In [53]:
 for msg in messages[:1]: 
        # Get the message from its id 
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    
    payload = txt['payload'] 
    headers = payload['headers']
    print(payload['headers'])

[{'name': 'Delivered-To', 'value': 'automation2826@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a5e:9517:0:0:0:0:0 with SMTP id r23csp351087ioj;        Wed, 13 Jan 2021 03:53:45 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'ABdhPJwvUA/glTdc7Omv8lV42erl6Ftk/54I9Oy5bj8Ygw6H8I0bexJqUqeLJYFCeJtbb0akGbqw'}, {'name': 'X-Received', 'value': 'by 2002:a5d:4f10:: with SMTP id c16mr2182201wru.398.1610538825220;        Wed, 13 Jan 2021 03:53:45 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1610538825; cv=none;        d=google.com; s=arc-20160816;        b=HIGnnoRnL67e01l0u859rvWsa6DmG1PzVVbm1UET8Xtg6DF2gyGW0a9B6oUr893Y5/         rBfiOtl47tNiozl0ILITAJGlGJsleudfEloQoA1vwlyyTnnKzD7h8udy2t6wZ7U82ttM         row6yXJ4Ip7dtGDZs4VnuW95GwGnhwqcF4PNjX1eYaSqi94rTwR5KPDkYgHY2yGRATn1         ChvMERjzBEacjOStt3yjtQW6tTRmAf8JzoY8zbeDKA9t9eRK/aI+vCkXfPPkyvgLT+sM         4VnTmyu/nbrbUfMn+wuTfhE/gZ2sVt3Sv4zgxaq1Ow8E1v66Nzya58iu00Y/UhyYTwGI         NkoQ=='}, {'name': 'ARC-Mes

In [54]:
for d in headers: 
    if d['name'] == 'Subject': 
        subject = d['value'] 
    if d['name'] == 'From': 
        sender = d['value'] 

In [55]:
subject

"Welcome back Rajesh, here's your login link 🔑"

In [56]:
sender

'Tickertape <notifications@tickertape.in>'

In [70]:
data=payload['body']['data'].replace("-","+").replace("_","/") 
decoded_data=base64.b64decode(data)

In [ ]:
soup = BeautifulSoup(decoded_data , "html.parser") 
body = soup.body() 
body